# Protein Design
---
In this notebook we are going to redesign the human Acetlserotonin O-methyltransferase (ASMT) with the UniProt id P46597. The goal is to keep the catalytic core of the protein constant while remodeling the scaffold of the protein. The protein we are redesigning is a methyl serotonin methyl transferase.

In [1]:
import sys
sys.path.append('../')
from analysis import pdb

ASMT_pdb = '../example_data/structures/ASMT.pdb'
ASMT_seq = pdb.to_fasta(ASMT_pdb).split('\n')[1]
ASMT_sse = pdb.get_sse(ASMT_pdb)
print(ASMT_seq)
print(ASMT_sse)
pdb.show(ASMT_pdb, color='confidence')

MGSSEDQAYRLLNDYANGFMVSQVLFAACELGVFDLLAEAPGPLDVAAVAAGVRASAHGTELLLDICVSLKLLKVETRGGKAFYRNTELSSDYLTTVSPTSQCSMLKYMGRTSYRCWGHLADAVREGRNQYLETFGVPAEELFTAIYRSEGERLQFMQALQEVWSVNGRSVLTAFDLSVFPLMCDLGGGAGALAKECMSLYPGCKITVFDIPEVVWTAKQHFSFQEEEQIDFQEGDFFKDPLPEADLYILARVLHDWADGKCSHLLERIYHTCKPGGGILVIESLLDEDRRGPLLTQLYSLNMLVQTEGQERTPTHYHMLLSSAGFRDFQFKKTGAIYDAILARK
cccaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaccccccaaaaaaaacccaaaaaaaaaaaaaaccbbbbbcccbbbbbbcccccccccccccccaaaaaaaaaacaaaaaaaaaaaaaccccccccccccccccaaaaaaccaaaaaaaaaaaaaaaaccaaaaaaccccccccbbbbcccccaaaaaaaaaacccbbbbbbbcaaaaaaaacccccccccbbbbbbccccccccccccbbbbccccccccaaaaaaaaaaaaaacccccbbbbbccccccccccaaaaaaaaaaaaaccccccccaaaaaaaaaaacccbbbbbbcccccbbbbbcc


/Users/jonathanfunk/miniforge3/envs/proteusAI/lib/python3.8/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Docking
----
At first we are going to dock the substrates and products into the protein using DiffDock to get inspect where the active site of the protein is. In this naïve example, we are constraining the positions of the atoms of the catalytic site and redesign the remaining amino acids. In order to dock the substrates into the protein we will create a csv file to schedule the job and then use the inference script of DiffDock to perform the docking.

In [2]:
import os
import pandas as pd

path_to_ligands = '../example_data/molecules/'
ligands = [f for f in os.listdir(path_to_ligands)]
ligand_paths = [os.path.join(path_to_ligands, lig) for lig in ligands]

dock_df = pd.DataFrame({
    'complex_name' : [f'ASMT_{lig}' for lig in ligands],
    'protein_path' : [ASMT_pdb for _ in ligands],
    'ligand_description': [lig for lig in ligand_paths],
    'protein_sequence': [None for _ in ligands]
})
dock_df.to_csv('../example_data/docking/docking.csv')
dock_df

,complex_name,protein_path,ligand_description,protein_sequence
0,ASMT_SAM_melatonin_transition_state.sdf,../example_data/structures/ASMT.pdb,../example_data/molecules/SAM_melatonin_transi...,None
1,ASMT_SAH.sdf,../example_data/structures/ASMT.pdb,../example_data/molecules/SAH.sdf,None
2,ASMT_SAM.sdf,../example_data/structures/ASMT.pdb,../example_data/molecules/SAM.sdf,None
3,ASMT_N-acetylserotonin.sdf,../example_data/structures/ASMT.pdb,../example_data/molecules/N-acetylserotonin.sdf,None
4,ASMT_melatonin.sdf,../example_data/structures/ASMT.pdb,../example_data/molecules/melatonin.sdf,None


## Docking results
---
The docking simulation is run with the inference script of DiffDock with the following options:

```
python3 -m inference --protein_ligand_csv ../example_data/docking/docking.csv \
--out_dir ../example_data/docking/ --inference_steps 20 \
--samples_per_complex 40 --batch_size 10 \
--actual_steps 18 --no_final_step_noise --save_visualisation
```

We can visualize the docking results by looking at an ensemble of the most confidents docked complexes. To do that we will filter out the top 10 docked substrates and products and visualize them using py3Dmol.

In [3]:
import re
pattern = r'rank[1]_'

dock_results_path = '../example_data/docking/'
dock_dirs = [os.path.join(dock_results_path, d) for d in os.listdir('../example_data/docking/') if d.endswith('sdf')]

docked_ligands = []
for d in dock_dirs:
    for f in os.listdir(d):
        
        if re.search(pattern, f) and f.endswith('sdf'):
            docked_ligands.append(os.path.join(d, f))

educts = [f for f in docked_ligands if ('melatonin' in f or 'SAM' in f) and 'transition' not in f]
products = [f for f in docked_ligands if ('acet' in f or 'SAH' in f)]

In [4]:
import py3Dmol

view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline', 'color':'black', 'width':0.1})

view.addModel(open(ASMT_pdb, 'r').read(), format='pdb')
Prot=view.getModel()

Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.4, 'color':'white'})

for lig in educts:
    view.addModel(open(lig,'r').read(),format='sdf')
    ref_m = view.getModel()
    ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

for lig in products:
    view.addModel(open(lig,'r').read(),format='sdf')
    ref_m = view.getModel()
    ref_m.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})


view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We can see that the substrate and product are likely flipped in the docked structure, because the carbon which is methylated is not in contact with the oxygen which is transferred from SAM to melatonin. However, if we dock a reasonable structure of the transition state, we might see the arrangement of the ligands during the reaction.

In the next steps we are going to filter out the residues which are in a certain distance to the docked ligands to determine the catalytically relevant residues.

In [5]:
transition_states = [f for f in docked_ligands if 'transition' in f]

In [6]:
import py3Dmol

view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline', 'color':'black', 'width':0.1})

view.addModel(open(ASMT_pdb, 'r').read(), format='pdb')
Prot=view.getModel()

Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.4, 'color':'white'})

for lig in transition_states:
    view.addModel(open(lig,'r').read(),format='sdf')
    ref_m = view.getModel()
    ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})


view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
melatonin = [f for f in educts if 'melatonin' in f and 'rank1' in f][0]
SAM = [f for f in educts if 'SAM' in f and 'rank1' in f][0]
transition_state = [f for f in transition_states if 'transition' in f and 'rank1' in f][0]

In [8]:
from analysis import interactions

contacts = interactions.mol_contacts([melatonin, SAM, transition_state], ASMT_pdb)

ASMT_atms = interactions.get_atom_array(ASMT_pdb)
contact_indices = []
for res_id in contacts:
    res_name = ASMT_atms.res_name[ASMT_atms.res_id == res_id][0]
    contact_indices.append(res_id-1)
    print(res_name.capitalize() + str(res_id))

Met105
Met109
Ser113
Phe143
Tyr147
Phe156
Leu160
Trp164
Gly187
Gly188
Gly189
Leu193
Asp210
Ile211
Val214
Gly235
Asp236
Phe237
Phe238
Ala251
Arg252
His255
Asp256
Trp257
Tyr299
Asn302
Met303
Tyr338


## Design and design constraints
---
In this step we are going to load the ProteinDesign class which is going to be used to redesign the protein. The protein design class has many global constraint options like maximum sequnece length, confidence of the structure, globularity of the structure or the sequence identity to the original template. Parameters of the class like the number of steps, the temperature during sampling and its degredation constant.

Furthermore, the user can define custom constraints for individual residues which will be taken into account during the sampling steps and when calculating the energy. Currently the following constraints are possible:

constraints on mutations:
- no mutations at all
- substitutions only

energy function:
- secondary 

In [10]:
from design import MCMC

mut_p = (0.6, 0.2, 0.2)

res_constraints = {'no_mut':contact_indices,
                   'all_atm':contact_indices
                  }

outdir = '../example_data/designs/ASMT/pdbs/'

Design = MCMC.ProteinDesign(native_seq=ASMT_seq,
                            constraints=res_constraints,
                            outdir=outdir
                           )

print(res_constraints)
print(Design)
#Design.run()

{'no_mut': [104, 108, 112, 142, 146, 155, 159, 163, 186, 187, 188, 192, 209, 210, 213, 234, 235, 236, 237, 250, 251, 254, 255, 256, 298, 301, 302, 337], 'all_atm': [104, 108, 112, 142, 146, 155, 159, 163, 186, 187, 188, 192, 209, 210, 213, 234, 235, 236, 237, 250, 251, 254, 255, 256, 298, 301, 302, 337]}
ProteusAI.MCMC.Design class: 
---------------------------------------
When Hallucination.run() sequences will be hallucinated using this seed sequence:

MGSSEDQAYRLLNDYANGFMVSQVLFAACELGVFDLLAEAPGPLDVAAVAAGVRASAHGTELLLDICVSLKLLKVETRGGKAFYRNTELSSDYLTTVSPTSQCSMLKYMGRTSYRCWGHLADAVREGRNQYLETFGVPAEELFTAIYRSEGERLQFMQALQEVWSVNGRSVLTAFDLSVFPLMCDLGGGAGALAKECMSLYPGCKITVFDIPEVVWTAKQHFSFQEEEQIDFQEGDFFKDPLPEADLYILARVLHDWADGKCSHLLERIYHTCKPGGGILVIESLLDEDRRGPLLTQLYSLNMLVQTEGQERTPTHYHMLLSSAGFRDFQFKKTGAIYDAILARK

The following variables were set:

variable	|value
----------------+-------------------
algorithm: 	|simulated_annealing
steps: 		|1000
n_traj: 	|16
mut_p: 		|(0.6, 0.2, 0.2)
T: 		|10.0
M: 		|0.

In [23]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from analysis import video

video.make('../example_data/designs/ASMT/pdbs/', '../example_data/designs/ASMT/pngs/', '../example_data/designs/ASMT/')

def simulation_hub(path: str, t: int, data: pd.DataFrame):
    
    pdb_path = os.path.join(path_to_simulation, 'pdbs')
    png_path = os.path.join(path_to_simulation, 'pngs')
    data_path = os.path.join(path_to_simulation, 'data')
    
    df = data.iloc[:, :-5]
    #df = data.iloc[:, :-3]
    x_lim = df.max().max() * 1.1
    _t = '{:04d}'.format(t)
    _design = mut = str(data['description'].iloc[t])
    img_path = os.path.join(png_path, _design + '.png')
    sns.set_style('white')
    T = data['T'].iloc[0]
    M = data['M'].iloc[0]
    iters = data['iteration']
    Ts = [T / (1 + M * i) for i in iters]
    total_energies = df.sum(axis=1).tolist()

    # Create the GridSpec with a 2x2 layout and different ratios
    fig = plt.figure(figsize=(12, 8))
    gs = GridSpec(2, 2, width_ratios=[2, 1], height_ratios=[2, 1])

    # Load the image and display it in the first subplot
    img = plt.imread(img_path)
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(img)
    ax1.set_axis_off()
    ax1.set_title(_design)

    # Plot the data in the second subplot
    ax2 = fig.add_subplot(gs[0, 1])
    df = df.iloc[t, :]
    sns.barplot(x=df.values, y=df.index)
    ax2.set_title('Individual weighted energies')
    ax2.set(xlim=(0, x_lim))

    # Plot the data in the third subplot
    ax3 = fig.add_subplot(gs[1, 0])
    sns.lineplot(x=iters, y=total_energies, ax=ax3)
    plt.scatter(iters[t], total_energies[t], marker="*", s=80)
    mut = str(data['mut'].iloc[t])
    #mut = "[mutation]"
    props = dict(boxstyle='round', facecolor='lightblue', alpha=0.3)
    ax3.text(0.95, 0.95, mut, transform=ax3.transAxes, fontsize=12,
            verticalalignment='top', horizontalalignment='right', bbox=props)
    ax3.set_title('Total energy')

    # Plot the data in the fourth subplot
    ax4 = fig.add_subplot(gs[1, 1])
    plt.scatter(iters[t], Ts[t], marker="*", s=80)
    sns.lineplot(x=iters, y=Ts, ax=ax4)
    ax4.set_title('Temperature')

    # Adjust the spacing between subplots
    fig.tight_layout()

    # Show the figure
    #plt.show()
    
path_to_simulation = '../example_data/designs/ASMT/'
data = pd.read_csv(os.path.join(path_to_simulation, 'data/energy_log.pdb'))
n_files = len(os.listdir(path_to_simulation+'pdbs'))

interact(simulation_hub, path=fixed(path_to_simulation), t=(0, n_files-2), data=fixed(data))

interactive(children=(IntSlider(value=36, description='t', max=72), Output()), _dom_classes=('widget-interact'…

<function __main__.simulation_hub(path: str, t: int, data: pandas.core.frame.DataFrame)>

In [21]:
data

,e_len x 0.01,e_identity x 0.1,e_pTMs x 1,e_mean_pLDDTs x 1,e_globularity x 0.001,e_sasa x 0.02,e_bb_coord x 0.02,e_all_atm x 0.15,iteration,T,M,mut,description
0,0.45,0.099710,0.102471,0.108497,0.162902,0.127728,0.003653,0.006315,0,10.0,0.1,sub:E149C,0001_design
1,0.44,0.099420,0.099496,0.114353,0.161528,0.127671,0.057133,0.009083,1,10.0,0.1,del:L69,0002_design
2,0.44,0.099130,0.099347,0.113141,0.161478,0.127977,0.057252,0.008765,2,10.0,0.1,sub:L340T,0003_design
3,0.44,0.098841,0.098158,0.113467,0.161416,0.127933,0.057134,0.009349,3,10.0,0.1,sub:F178E,0004_design
4,0.44,0.098551,0.097902,0.113017,0.161175,0.127866,0.057581,0.009196,4,10.0,0.1,sub:V32C,0005_design
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0.45,0.084527,0.082588,0.117432,0.165357,0.128750,0.044163,0.056104,60,10.0,0.1,sub:Y61S,0061_design
61,0.45,0.084527,0.082659,0.116960,0.165270,0.128558,0.044562,0.056096,61,10.0,0.1,sub:K9T,0062_design
62,0.45,0.084241,0.082835,0.116724,0.165421,0.128474,0.044244,0.056263,62,10.0,0.1,sub:A39T,0063_design
63,0.45,0.084241,0.082835,0.116724,0.165421,0.128474,0.044244,0.056263,63,10.0,0.1,sub:L232L,0064_design


In [22]:
from IPython.display import HTML

# Replace these file paths with your own
gif1 = "../example_data/designs/ASMT/design.gif"

# Create the HTML code to display the GIFs in a 2x2 grid
html = f"""
<div style="display: flex; flex-wrap: wrap; justify-content: space-between;">
    <img src="{gif1}" style="width: 20%; margin-bottom: 10px;">
</div>
"""

# Display the HTML code
HTML(html)